In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!



**Files needed to run:**

*   secondhalf_AZ.[shp,dbf,prj,shx,cpg] --> Merged catchment area with dam attributes
*   dams_test_[1-3].csv --> Intermediate output to combine


# Set up workspace

In [ ]:
import ee
from collections import Counter
import pandas as pd
import math
from google.colab import files
import csv
import fiona

In [ ]:
!pip install geopandas
!pip install descartes

     |████████████████████████████████| 1.0MB 2.8MB/s 
     |████████████████████████████████| 15.3MB 485kB/s 
     |████████████████████████████████| 6.5MB 46.3MB/s 


In [ ]:
import geopandas as gpd

In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=BK9DdZCMCLxLZ54Y6HxaxbUVuCAFIb9S0OJy5eV_Nu4&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g4pyjYbooEKRip3m3QJkBcSxXw_9HyeMleNw3_IFK8JVCDlHaR0EvU

Successfully saved authorization token.


# Load data

In [ ]:
#import GSW yearly data
GSW_yearly = ee.ImageCollection('JRC/GSW1_1/YearlyHistory')

In [ ]:
#define a function to remove no-data pixels
def nodata_mask(image):
  qa = image.select('waterClass')
  no_data = qa.neq(0)
  return image.updateMask(no_data)

In [ ]:
#remove all no-data pixels
yearly_processed = GSW_yearly.map(nodata_mask)

In [ ]:
#open catchment shapefile data (upload all files in '~/Analysis/catchment areas/merged_results' files first)
catchments = gpd.read_file('secondhalf_AZ.shp')
catchments = catchments.sort_values('Feature_ID', ascending= True).reset_index(drop = True)
catchments


,Feature_ID,Dam_Name,Alt_Name,Reservoir,Alt_Reserv,River,Alt_River,Main_basin,Sub_basin,Continent,Admin0,Admin1,Admin2,Admin3,Sec_Admin0,Sec_Admin1,Sec_Admin2,Seco_Admin,Year_Fin,Year_Const,Alt_Yr_Fin,Height,Alt_Height,Length,Alt_Length,Volume_Con,Volume_Rep,Volume_Max,Volume_Min,Area_Con,Area_Pol,Area_Rep,Area_Max,Area_Min,Avg_Depth,Sediment_r,P_Irrig,P_Hydro,P_WSupp,P_FCont,P_Navig,P_Fishr,P_Livst,P_Recrn,P_PCont,P_WStor,P_Other,Main_P,Sec_P,Mult_Dams,Timeline,Lat,Long,Flag_Qual,Capac,Alt_Capac,Source,Comment,Editor,Main_P_Map,intp_dist,Intp_Coord,GDAT_Coord,Correct_Co,area,geometry
0,6367,Teesta,None,None,None,Teesta,None,Brahmaputra,None,Asia,Bangladesh,None,None,None,None,None,None,None,None,None,None,96,None,None,None,NaN,NaN,NaN,None,NaN,NaN,None,None,None,NaN,None,1.0,1,None,None,None,None,None,None,None,None,None,Hydroelectricity,None,None,None,26.177083,89.060417,1,None,None,GRanD,None,Xinyi,Hydroelectricity,2.553000e-12,POINT (89.0604169999978410 26.1770830000013603),POINT (89.0604170000000011 26.1770829999999997),POINT (89.0604169999978410 26.1770830000013603),1.415903e+08,"POLYGON ((88.88166 26.37694, 88.89912 26.37694..."
1,6369,Punatsangchhu,Punatsangchu-I,None,None,Punatsangchu,None,None,None,Asia,Bhutan,Wangdue Dzongkhag,Wangdue phodrang town,None,None,None,None,None,None,None,None,141,None,None,None,NaN,NaN,NaN,None,NaN,NaN,None,None,None,NaN,None,NaN,1,None,None,None,None,None,None,None,None,None,Hydroelectricity,None,None,None,27.255654,90.042538,1,1200,None,"AQUASTAT, Embassy of India in Thimphu, Punatsa...",None,Xinyi,Hydroelectricity,1.278449e-03,POINT (90.0434419999987483 27.2565580000002790),POINT (90.0425379999999933 27.2556539999999998),POINT (90.0434419999987483 27.2565580000002790),4.176132e+08,"POLYGON ((89.99867 27.45904, 90.05103 27.45904..."
2,6370,Punatsangchhu-II,None,None,None,None,None,None,None,Asia,Bhutan,None,None,None,None,None,None,None,None,None,None,86,None,None,None,NaN,NaN,NaN,None,NaN,NaN,None,None,None,NaN,None,NaN,1,None,None,None,None,None,None,None,None,None,Hydroelectricity,None,None,None,27.421057,89.904465,1,1020,None,"Embassy of India in Thimphu, Punatsangchhu-II ...",None,Xinyi,Hydroelectricity,1.785000e-03,POINT (89.9062499999994600 27.4210569999967149),POINT (89.9044650000000019 27.4210570000000011),POINT (89.9062499999994600 27.4210569999967149),2.002803e+07,"POLYGON ((89.82414 27.42414, 89.91141 27.42414..."
3,6376,Dagachhu,None,None,None,Dagachhu,None,None,None,Asia,Bhutan,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,NaN,NaN,None,None,None,NaN,None,NaN,1,None,None,None,None,None,None,None,None,None,Hydroelectricity,None,None,None,27.036496,89.919902,1,126,None,"Druk Green, Dagachhu Hydro Power Corporation L...",None,Xinyi,Hydroelectricity,1.368487e-03,POINT (89.9208696666636342 27.0374636666687778),POINT (89.9199019999999933 27.0364959999999996),POINT (89.9208696666636342 27.0374636666687778),6.698913e+06,"MULTIPOLYGON (((89.89395 27.02271, 89.91141 27..."
4,6377,Nikachhu,None,None,None,None,None,None,None,Asia,Bhutan,Trongsa Dzongkhag,Tangsibji Gewog,Lorim,None,None,None,None,None,None,None,38,None,90.25,None,NaN,0.307,0.537,None,NaN,NaN,None,None,None,NaN,None,NaN,1,None,None,None,None,None,None,None,None,None,Hydroelectricity,None,None,None,27.448546,90.374313,1,118,None,Tangsibji Hydro Energy Ltd (https://www.thye.bt/),Water from power generation is discharged to M...,Xinyi,Hydroelectricity,6.293333e-04,POINT (90.3743129999993187 27.4479166666662380),POINT (90.3743130000000008 27.4485460000000003),POINT (90.3743129999993187 27.4479166666662380),3.667583e+08,"POLYGON ((90.27793 27.72084, 90.33029 27.72084..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4173,35111,Zakucac Hydroelectric Power Plant Croati

In [ ]:
catchments_proj = catchments.to_crs("EPSG:3395")

In [ ]:
catchments_proj.crs

<Projected CRS: EPSG:3395>
Name: WGS 84 / World Mercator
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World between 80°S and 84°N.
- bounds: (-180.0, -80.0, 180.0, 84.0)
Coordinate Operation:
- name: World Mercator
- method: Mercator (variant A)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
catchments_proj['proj_area'] = catchments_proj.area

In [ ]:
catchments_normal = catchments[catchments_proj.proj_area < 3.0e11]

In [ ]:
catchments_large_AZ = catchments[catchments_proj.proj_area >= 3.0e11]

In [ ]:
catchments_large_AZ.to_file("catchments_large_AZ.shp")

In [ ]:
catchments_normal

,Feature_ID,Dam_Name,Alt_Name,Reservoir,Alt_Reserv,River,Alt_River,Main_basin,Sub_basin,Continent,Admin0,Admin1,Admin2,Admin3,Sec_Admin0,Sec_Admin1,Sec_Admin2,Seco_Admin,Year_Fin,Year_Const,Alt_Yr_Fin,Height,Alt_Height,Length,Alt_Length,Volume_Con,Volume_Rep,Volume_Max,Volume_Min,Area_Con,Area_Pol,Area_Rep,Area_Max,Area_Min,Avg_Depth,Sediment_r,P_Irrig,P_Hydro,P_WSupp,P_FCont,P_Navig,P_Fishr,P_Livst,P_Recrn,P_PCont,P_WStor,P_Other,Main_P,Sec_P,Mult_Dams,Timeline,Lat,Long,Flag_Qual,Capac,Alt_Capac,Source,Comment,Editor,Main_P_Map,intp_dist,Intp_Coord,GDAT_Coord,Correct_Co,area,geometry
0,6367,Teesta,None,None,None,Teesta,None,Brahmaputra,None,Asia,Bangladesh,None,None,None,None,None,None,None,None,None,None,96,None,None,None,NaN,NaN,NaN,None,NaN,NaN,None,None,None,NaN,None,1.0,1,None,None,None,None,None,None,None,None,None,Hydroelectricity,None,None,None,26.177083,89.060417,1,None,None,GRanD,None,Xinyi,Hydroelectricity,2.553000e-12,POINT (89.0604169999978410 26.1770830000013603),POINT (89.0604170000000011 26.1770829999999997),POINT (89.0604169999978410 26.1770830000013603),1.415903e+08,"POLYGON ((88.88166 26.37694, 88.89912 26.37694..."
1,6369,Punatsangchhu,Punatsangchu-I,None,None,Punatsangchu,None,None,None,Asia,Bhutan,Wangdue Dzongkhag,Wangdue phodrang town,None,None,None,None,None,None,None,None,141,None,None,None,NaN,NaN,NaN,None,NaN,NaN,None,None,None,NaN,None,NaN,1,None,None,None,None,None,None,None,None,None,Hydroelectricity,None,None,None,27.255654,90.042538,1,1200,None,"AQUASTAT, Embassy of India in Thimphu, Punatsa...",None,Xinyi,Hydroelectricity,1.278449e-03,POINT (90.0434419999987483 27.2565580000002790),POINT (90.0425379999999933 27.2556539999999998),POINT (90.0434419999987483 27.2565580000002790),4.176132e+08,"POLYGON ((89.99867 27.45904, 90.05103 27.45904..."
2,6370,Punatsangchhu-II,None,None,None,None,None,None,None,Asia,Bhutan,None,None,None,None,None,None,None,None,None,None,86,None,None,None,NaN,NaN,NaN,None,NaN,NaN,None,None,None,NaN,None,NaN,1,None,None,None,None,None,None,None,None,None,Hydroelectricity,None,None,None,27.421057,89.904465,1,1020,None,"Embassy of India in Thimphu, Punatsangchhu-II ...",None,Xinyi,Hydroelectricity,1.785000e-03,POINT (89.9062499999994600 27.4210569999967149),POINT (89.9044650000000019 27.4210570000000011),POINT (89.9062499999994600 27.4210569999967149),2.002803e+07,"POLYGON ((89.82414 27.42414, 89.91141 27.42414..."
3,6376,Dagachhu,None,None,None,Dagachhu,None,None,None,Asia,Bhutan,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,NaN,NaN,None,None,None,NaN,None,NaN,1,None,None,None,None,None,None,None,None,None,Hydroelectricity,None,None,None,27.036496,89.919902,1,126,None,"Druk Green, Dagachhu Hydro Power Corporation L...",None,Xinyi,Hydroelectricity,1.368487e-03,POINT (89.9208696666636342 27.0374636666687778),POINT (89.9199019999999933 27.0364959999999996),POINT (89.9208696666636342 27.0374636666687778),6.698913e+06,"MULTIPOLYGON (((89.89395 27.02271, 89.91141 27..."
4,6377,Nikachhu,None,None,None,None,None,None,None,Asia,Bhutan,Trongsa Dzongkhag,Tangsibji Gewog,Lorim,None,None,None,None,None,None,None,38,None,90.25,None,NaN,0.307,0.537,None,NaN,NaN,None,None,None,NaN,None,NaN,1,None,None,None,None,None,None,None,None,None,Hydroelectricity,None,None,None,27.448546,90.374313,1,118,None,Tangsibji Hydro Energy Ltd (https://www.thye.bt/),Water from power generation is discharged to M...,Xinyi,Hydroelectricity,6.293333e-04,POINT (90.3743129999993187 27.4479166666662380),POINT (90.3743130000000008 27.4485460000000003),POINT (90.3743129999993187 27.4479166666662380),3.667583e+08,"POLYGON ((90.27793 27.72084, 90.33029 27.72084..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4173,35111,Zakucac Hydroelectric Power Plant Croati

In [ ]:
catchments_0_2000 = catchments_normal.iloc[0:2000,]
catchments_2000_end = catchments_normal.iloc[2000:len(catchments_normal),]

In [ ]:
years = list(range(1984,2019)) #list of all years with data in GSW
years

[1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018]

In [ ]:
catchments.iloc[1621]

# Create functions to extract water pixels

In [ ]:
def get_catchment_data(dam): #obtain catchment area of dam as GEE object
  geom = dam['geometry']
  catchment_geojson = gpd.GeoSeries(geom).__geo_interface__
  catchment_geom = catchment_geojson['features'][0]
  fc = ee.FeatureCollection([ee.Feature(catchment_geom)])
  return fc

def get_year_counts(year, fc): #obtain pixel counts of all years for a dam
  this_year = str(year)
  next_year = str(year + 1)
  year_image = ee.Image(yearly_processed.filterDate(this_year,next_year).first())
  year_result = year_image.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, True, 1e12).get('waterClass')
  year_dict = dict(Counter(year_result.getInfo()))
  year_count = sorted(list(year_dict.items()), key = lambda x: x[0])
  return year_count

def check_year_counts(year_count): #organize pixel counts to add to results dataframe
  count_indices = [x[0] for x in year_count]
  missing_indices = list(set([1,2,3]) - set(count_indices))
  for i in missing_indices:
    empty_set_index = (i, 0)
    year_count.append(empty_set_index)
  #sort
  year_count = sorted(year_count, key = lambda x: x[0])
  list_year_count = [x[1] for x in year_count]
  return list_year_count


In [ ]:
#name all columns of results dataframe
excel_columns = ['actual_yr','feature_id',
                 '1984_1','1984_2','1984_3',
                 '1985_1','1985_2','1985_3',
                 '1986_1','1986_2','1986_3',
                 '1987_1','1987_2','1987_3',
                 '1988_1','1988_2','1988_3',
                 '1989_1','1989_2','1989_3',
                 '1990_1','1990_2','1990_3',
                 '1991_1','1991_2','1991_3',
                 '1992_1','1992_2','1992_3',
                 '1993_1','1993_2','1993_3',
                 '1994_1','1994_2','1994_3',
                 '1995_1','1995_2','1995_3',
                 '1996_1','1996_2','1996_3',
                 '1997_1','1997_2','1997_3',
                 '1998_1','1998_2','1998_3',
                 '1999_1','1999_2','1999_3',
                 '2000_1','2000_2','2000_3',
                 '2001_1','2001_2','2001_3',
                 '2002_1','2002_2','2002_3',
                 '2003_1','2003_2','2003_3',
                 '2004_1','2004_2','2004_3',
                 '2005_1','2005_2','2005_3',
                 '2006_1','2006_2','2006_3',
                 '2007_1','2007_2','2007_3',
                 '2008_1','2008_2','2008_3',
                 '2009_1','2009_2','2009_3',
                 '2010_1','2010_2','2010_3',
                 '2011_1','2011_2','2011_3',
                 '2012_1','2012_2','2012_3',
                 '2013_1','2013_2','2013_3',
                 '2014_1','2014_2','2014_3',
                 '2015_1','2015_2','2015_3',
                 '2016_1','2016_2','2016_3',
                 '2017_1','2017_2','2017_3',
                 '2018_1','2018_2','2018_3']


In [ ]:
catchments_normal[catchments_normal.Feature_ID == 6753]

In [ ]:
#Debug: Check that main code works
test = 1
fc = get_catchment_data(catchments_normal.iloc[test])
dam_excel_row = [catchments_normal.iloc[test]['Year_Fin'], catchments_normal.iloc[1]['Feature_ID']] #set up dataframe row for dam

for year in years:
    year_count = get_year_counts(year, fc)
    list_year_count = check_year_counts(year_count) #correctly order results for each year
    dam_excel_row = dam_excel_row + list_year_count #put all years together into a row

catchments_normal.iloc[test]['Year_Fin']
catchments_normal.iloc[test]['Feature_ID']

6369

# Execute code on all dams with completion year (training data)

In [ ]:
#THIS IS THE MAIN EXEC CODE

#df = pd.DataFrame(columns = excel_columns) #create dataframe of results

for index, row in catchments_1_2000.iterrows(): #iterate over each dam
  fc = get_catchment_data(row) #get catchment area of dam as GEE object
  dam_excel_row = [row['Year_Fin'], row['Feature_ID']] #set up dataframe row for dam

  for year in years:
    year_count = get_year_counts(year, fc) #get pixel counts for each year
    list_year_count = check_year_counts(year_count) #correctly order results for each year
    dam_excel_row = dam_excel_row + list_year_count #put all years together into a row

  with open('dams_test_1.csv', 'a') as file:
    writer = csv.writer(file)
    writer.writerow(dam_excel_row)

  #add_df = pd.DataFrame([dam_excel_row], columns = excel_columns) #prepare row to be added to df
  #df = pd.concat([df, add_df]) #append row to the results dataframe
  print('dam ' + str(index) + ' (' + str(row['Feature_ID']) + ') completed') #track progress

files.download('dams_test_1.csv')

In [ ]:
files.download('dams_test_1.csv')

In [ ]:
#THIS IS THE MAIN EXEC CODE

#df = pd.DataFrame(columns = excel_columns) #create dataframe of results

for index, row in catchments_normal.iterrows(): #iterate over each dam
  fc = get_catchment_data(row) #get catchment area of dam as GEE object
  dam_excel_row = [row['Year_Fin'], row['Feature_ID']] #set up dataframe row for dam

  for year in years:
    year_count = get_year_counts(year, fc) #get pixel counts for each year
    list_year_count = check_year_counts(year_count) #correctly order results for each year
    dam_excel_row = dam_excel_row + list_year_count #put all years together into a row

  with open('dams_test_normal.csv', 'a') as file:
    writer = csv.writer(file)
    writer.writerow(dam_excel_row)

  #add_df = pd.DataFrame([dam_excel_row], columns = excel_columns) #prepare row to be added to df
  #df = pd.concat([df, add_df]) #append row to the results dataframe
  print('dam ' + str(index) + ' (' + str(row['Feature_ID']) + ') completed') #track progress

files.download('dams_test_normal.csv')

dam 0 (6367) completed
dam 1 (6369) completed
dam 2 (6370) completed
dam 3 (6376) completed
dam 4 (6377) completed
dam 5 (6378) completed
dam 6 (6379) completed
dam 7 (6380) completed
dam 8 (6382) completed
dam 9 (6383) completed
dam 10 (6384) completed
dam 11 (6385) completed
dam 12 (6386) completed
dam 13 (6387) completed
dam 14 (6389) completed
dam 15 (6390) completed
dam 16 (6391) completed
dam 17 (6393) completed
dam 18 (6394) completed
dam 19 (6395) completed
dam 20 (6396) completed
dam 21 (6397) completed
dam 22 (6398) completed
dam 23 (6399) completed
dam 24 (6400) completed
dam 25 (6401) completed
dam 26 (6495) completed
dam 27 (6504) completed
dam 28 (6505) completed
dam 29 (6518) completed
dam 30 (6519) completed
dam 31 (6520) completed
dam 32 (6521) completed
dam 33 (6523) completed
dam 34 (6526) completed
dam 35 (6531) completed
dam 36 (6532) completed
dam 37 (6534) completed
dam 38 (6536) completed
dam 39 (6537) completed
dam 40 (6539) completed
dam 41 (6544) completed
da

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#export dataframe
#df.to_csv('dams_training.csv')
#df.to_excel('dams_training.xlsx')
#files.download('dams_training.csv')
#files.download('dams_training.xlsx')

In [ ]:
for index, row in catchments_2000_4000.iterrows(): #iterate over each dam
  fc = get_catchment_data(row) #get catchment area of dam as GEE object
  dam_excel_row = [row['Year_Fin'], row['Feature_ID']] #set up dataframe row for dam

  for year in years:
    year_count = get_year_counts(year, fc) #get pixel counts for each year
    list_year_count = check_year_counts(year_count) #correctly order results for each year
    dam_excel_row = dam_excel_row + list_year_count #put all years together into a row

  with open('dams_training_2.csv', 'a') as file:
    writer = csv.writer(file)
    writer.writerow(dam_excel_row)

  #add_df = pd.DataFrame([dam_excel_row], columns = excel_columns) #prepare row to be added to df
  #df = pd.concat([df, add_df]) #append row to the results dataframe
  print('dam ' + str(index) + ' (' + str(row['Feature_ID']) + ') completed') #track progress

files.download('dams_training_2.csv')

In [ ]:
for index, row in catchments_4000_5484.iterrows(): #iterate over each dam
  fc = get_catchment_data(row) #get catchment area of dam as GEE object
  dam_excel_row = [row['Year_Fin'], row['Feature_ID']] #set up dataframe row for dam

  for year in years:
    year_count = get_year_counts(year, fc) #get pixel counts for each year
    list_year_count = check_year_counts(year_count) #correctly order results for each year
    dam_excel_row = dam_excel_row + list_year_count #put all years together into a row

  with open('dams_training_3.csv', 'a') as file:
    writer = csv.writer(file)
    writer.writerow(dam_excel_row)

  #add_df = pd.DataFrame([dam_excel_row], columns = excel_columns) #prepare row to be added to df
  #df = pd.concat([df, add_df]) #append row to the results dataframe
  print('dam ' + str(index) + ' (' + str(row['Feature_ID']) + ') completed') #track progress

files.download('dams_training_3.csv')

# Save training data

In [ ]:
# Append all excel files
with open('dams_training_all.csv', 'a') as fout:
    writer = csv.writer(fout)
    writer.writerow(excel_columns)
    
    for num in range(1,4):
      f = open("dams_training_"+str(num)+".csv")
      #f.next() # skip the header
      for line in f:
        fout.write(line)
      f.close() # not really needed

    fout.close()

# Export data
files.download('dams_training_all.csv')

In [ ]:
#CODE ENDS HERE

# (Test code)

In [ ]:
#ALL LINES BELOW ARE TEST CODES

In [ ]:
sample_dam = catchments.iloc[0]
geom = sample_dam['geometry']
catchment_geojson = gpd.GeoSeries(geom).__geo_interface__
catchment_geom = catchment_geojson['features'][0]

fc = ee.FeatureCollection([ee.Feature(catchment_geom)])

In [ ]:
sample_dam

In [ ]:
year_1984 = ee.Image(yearly_processed.filterDate('1984','1985').first())
year_1985 = ee.Image(yearly_processed.filterDate('1985','1986').first())
year_1986 = ee.Image(yearly_processed.filterDate('1986','1987').first())
year_1987 = ee.Image(yearly_processed.filterDate('1987','1988').first())
year_1988 = ee.Image(yearly_processed.filterDate('1988','1989').first())
year_1989 = ee.Image(yearly_processed.filterDate('1989','1990').first())
year_1990 = ee.Image(yearly_processed.filterDate('1990','1991').first())
year_1991 = ee.Image(yearly_processed.filterDate('1991','1992').first())
year_1992 = ee.Image(yearly_processed.filterDate('1992','1993').first())
year_1993 = ee.Image(yearly_processed.filterDate('1993','1994').first())
year_1994 = ee.Image(yearly_processed.filterDate('1994','1995').first())
year_1995 = ee.Image(yearly_processed.filterDate('1995','1996').first())
year_1996 = ee.Image(yearly_processed.filterDate('1996','1997').first())
year_1997 = ee.Image(yearly_processed.filterDate('1997','1998').first())
year_1998 = ee.Image(yearly_processed.filterDate('1998','1999').first())
year_1999 = ee.Image(yearly_processed.filterDate('1999','2000').first())
year_2000 = ee.Image(yearly_processed.filterDate('2000','2001').first())
year_2001 = ee.Image(yearly_processed.filterDate('2001','2002').first())
year_2002 = ee.Image(yearly_processed.filterDate('2002','2003').first())
year_2003 = ee.Image(yearly_processed.filterDate('2003','2004').first())
year_2004 = ee.Image(yearly_processed.filterDate('2004','2005').first())
year_2005 = ee.Image(yearly_processed.filterDate('2005','2006').first())
year_2006 = ee.Image(yearly_processed.filterDate('2006','2007').first())
year_2007 = ee.Image(yearly_processed.filterDate('2007','2008').first())
year_2008 = ee.Image(yearly_processed.filterDate('2008','2009').first())
year_2009 = ee.Image(yearly_processed.filterDate('2009','2010').first())
year_2010 = ee.Image(yearly_processed.filterDate('2010','2011').first())
year_2011 = ee.Image(yearly_processed.filterDate('2011','2012').first())
year_2012 = ee.Image(yearly_processed.filterDate('2012','2013').first())
year_2013 = ee.Image(yearly_processed.filterDate('2013','2014').first())
year_2014 = ee.Image(yearly_processed.filterDate('2014','2015').first())
year_2015 = ee.Image(yearly_processed.filterDate('2015','2016').first())
year_2016 = ee.Image(yearly_processed.filterDate('2016','2017').first())
year_2017 = ee.Image(yearly_processed.filterDate('2017','2018').first())
year_2018 = ee.Image(yearly_processed.filterDate('2018','2019').first())
#year_2019 = ee.Image(yearly_processed.filterDate('2019','2020').first())

In [ ]:
result_1984 = year_1984.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1984_dict = dict(Counter(result_1984.getInfo()))
result_1984_count = sorted(list(result_1984_dict.items()), key = lambda x: x[0])

result_1985 = year_1985.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1985_dict = dict(Counter(result_1985.getInfo()))
result_1985_count = sorted(list(result_1985_dict.items()), key = lambda x: x[0])

result_1986 = year_1986.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1986_dict = dict(Counter(result_1986.getInfo()))
result_1986_count = sorted(list(result_1986_dict.items()), key = lambda x: x[0])

result_1987 = year_1987.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1987_dict = dict(Counter(result_1987.getInfo()))
result_1987_count = sorted(list(result_1987_dict.items()), key = lambda x: x[0])

result_1988 = year_1988.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1988_dict = dict(Counter(result_1988.getInfo()))
result_1988_count = sorted(list(result_1988_dict.items()), key = lambda x: x[0])

result_1989 = year_1989.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1989_dict = dict(Counter(result_1989.getInfo()))
result_1989_count = sorted(list(result_1989_dict.items()), key = lambda x: x[0])

result_1990 = year_1990.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1990_dict = dict(Counter(result_1990.getInfo()))
result_1990_count = sorted(list(result_1990_dict.items()), key = lambda x: x[0])

result_1991 = year_1991.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1991_dict = dict(Counter(result_1991.getInfo()))
result_1991_count = sorted(list(result_1991_dict.items()), key = lambda x: x[0])

result_1992 = year_1992.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1992_dict = dict(Counter(result_1992.getInfo()))
result_1992_count = sorted(list(result_1992_dict.items()), key = lambda x: x[0])

result_1993 = year_1993.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1993_dict = dict(Counter(result_1993.getInfo()))
result_1993_count = sorted(list(result_1993_dict.items()), key = lambda x: x[0])

result_1994 = year_1994.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1994_dict = dict(Counter(result_1994.getInfo()))
result_1994_count = sorted(list(result_1994_dict.items()), key = lambda x: x[0])

result_1995 = year_1995.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1995_dict = dict(Counter(result_1995.getInfo()))
result_1995_count = sorted(list(result_1995_dict.items()), key = lambda x: x[0])

result_1996 = year_1996.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1996_dict = dict(Counter(result_1996.getInfo()))
result_1996_count = sorted(list(result_1996_dict.items()), key = lambda x: x[0])

result_1997 = year_1997.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1997_dict = dict(Counter(result_1997.getInfo()))
result_1997_count = sorted(list(result_1997_dict.items()), key = lambda x: x[0])

result_1998 = year_1998.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1998_dict = dict(Counter(result_1998.getInfo()))
result_1998_count = sorted(list(result_1998_dict.items()), key = lambda x: x[0])

result_1999 = year_1999.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1999_dict = dict(Counter(result_1999.getInfo()))
result_1999_count = sorted(list(result_1999_dict.items()), key = lambda x: x[0])

result_2000 = year_2000.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2000_dict = dict(Counter(result_2000.getInfo()))
result_2000_count = sorted(list(result_2000_dict.items()), key = lambda x: x[0])

result_2001 = year_2001.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2001_dict = dict(Counter(result_2001.getInfo()))
result_2001_count = sorted(list(result_2001_dict.items()), key = lambda x: x[0])

result_2002 = year_2002.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2002_dict = dict(Counter(result_2002.getInfo()))
result_2002_count = sorted(list(result_2002_dict.items()), key = lambda x: x[0])

result_2003 = year_2003.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2003_dict = dict(Counter(result_2003.getInfo()))
result_2003_count = sorted(list(result_2003_dict.items()), key = lambda x: x[0])

result_2004 = year_2004.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2004_dict = dict(Counter(result_2004.getInfo()))
result_2004_count = sorted(list(result_2004_dict.items()), key = lambda x: x[0])

result_2005 = year_2005.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2005_dict = dict(Counter(result_2005.getInfo()))
result_2005_count = sorted(list(result_2005_dict.items()), key = lambda x: x[0])

result_2006 = year_2006.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2006_dict = dict(Counter(result_2006.getInfo()))
result_2006_count = sorted(list(result_2006_dict.items()), key = lambda x: x[0])

result_2007 = year_2007.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2007_dict = dict(Counter(result_2007.getInfo()))
result_2007_count = sorted(list(result_2007_dict.items()), key = lambda x: x[0])

result_2008 = year_2008.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2008_dict = dict(Counter(result_2008.getInfo()))
result_2008_count = sorted(list(result_2008_dict.items()), key = lambda x: x[0])

result_2009 = year_2009.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2009_dict = dict(Counter(result_2009.getInfo()))
result_2009_count = sorted(list(result_2009_dict.items()), key = lambda x: x[0])

result_2010 = year_2010.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2010_dict = dict(Counter(result_2010.getInfo()))
result_2010_count = sorted(list(result_2010_dict.items()), key = lambda x: x[0])

result_2011 = year_2011.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2011_dict = dict(Counter(result_2011.getInfo()))
result_2011_count = sorted(list(result_2011_dict.items()), key = lambda x: x[0])

result_2012 = year_2012.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2012_dict = dict(Counter(result_2012.getInfo()))
result_2012_count = sorted(list(result_2012_dict.items()), key = lambda x: x[0])

result_2013 = year_2013.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2013_dict = dict(Counter(result_2013.getInfo()))
result_2013_count = sorted(list(result_2013_dict.items()), key = lambda x: x[0])

result_2014 = year_2014.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2014_dict = dict(Counter(result_2014.getInfo()))
result_2014_count = sorted(list(result_2014_dict.items()), key = lambda x: x[0])

result_2015 = year_2015.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2015_dict = dict(Counter(result_2015.getInfo()))
result_2015_count = sorted(list(result_2015_dict.items()), key = lambda x: x[0])

result_2016 = year_2016.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2016_dict = dict(Counter(result_2016.getInfo()))
result_2016_count = sorted(list(result_2016_dict.items()), key = lambda x: x[0])

result_2017 = year_2017.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2017_dict = dict(Counter(result_2017.getInfo()))
result_2017_count = sorted(list(result_2017_dict.items()), key = lambda x: x[0])

result_2018 = year_2018.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2018_dict = dict(Counter(result_2018.getInfo()))
result_2018_count = sorted(list(result_2018_dict.items()), key = lambda x: x[0])

#result_2019 = year_2019.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
#result_2019_dict = dict(Counter(result_2019.getInfo()))
#result_2019_count = sorted(list(result_2019_dict.items()), key = lambda x: x[0])

In [ ]:
result_2019 = year_2019.reduceRegion(ee.Reducer.toList(), fc, 30).get('waterClass')
result_2019_dict = dict(Counter(result_2019.getInfo()))
result_2019_count = sorted(list(result_2019_dict.items()), key = lambda x: x[0])

In [ ]:
print(1984, result_1984_count)
print(1985, result_1985_count)
print(1986, result_1986_count)
print(1987, result_1987_count)
print(1988, result_1988_count)
print(1989, result_1989_count)
print(1990, result_1990_count)
print(1991, result_1991_count)
print(1992, result_1992_count)
print(1993, result_1993_count)
print(1994, result_1994_count)
print(1995, result_1995_count)
print(1996, result_1996_count)
print(1997, result_1997_count)
print(1998, result_1998_count)
print(1999, result_1999_count)
print(2000, result_2000_count)
print(2001, result_2001_count)
print(2002, result_2002_count)
print(2003, result_2003_count)
print(2004, result_2004_count)
print(2005, result_2005_count)
print(2006, result_2006_count)
print(2007, result_2007_count)
print(2008, result_2008_count)
print(2009, result_2009_count)
print(2010, result_2010_count)
print(2011, result_2011_count)
print(2012, result_2012_count)
print(2013, result_2013_count)
print(2014, result_2014_count)
print(2015, result_2015_count)
print(2016, result_2016_count)
print(2017, result_2017_count)
print(2018, result_2018_count)

In [ ]:
[x[0] for x in result_2018_count]

In [ ]:
#sum up the 2's and 3's to get count of water pixels and no-water pixels (1)
#and then look at the years with the biggest changes or when the changes begin to occur

In [ ]:
#Dam #2468 (Cabeça de boi, Apiacás river) — a series of three run-of-river projects
#Complications: one part of a hydroelectric complex that started construction in 2014
#and completed in 2016
#Since it is run-of-the-river, there might not be that much storage caused by the
#dam, and hence water-changes might not be very significant.
#2469 (da Fazenda) is also part of the complex

In [ ]:
#loop through all of the dams — maintain current structure of code
#but automate the process to go through all rows of the shapefile
#create a dataframe with dams as rows and years*pixels for columns: three columns per year
#i.e. 1984-1, 1984-2, 1984-3 — don't add the 2's and 3's yet

In [ ]:
sample_dam = catchments.iloc[0]
geom = sample_dam['geometry']
catchment_geojson = gpd.GeoSeries(geom).__geo_interface__
catchment_geom = catchment_geojson['features'][0]

fc = ee.FeatureCollection([ee.Feature(catchment_geom)])

before_start = row['pre_init_f']
before_end = row['pre_end']
after_start = row['post_init']
after_end = row['post_end_f']

range_predam = yearly_processed.filterDate(before_start,before_end)
range_postdam = yearly_processed.filterDate(after_start,after_end)

median_predam = range_predam.reduce(ee.Reducer.median());
median_postdam = range_postdam.reduce(ee.Reducer.median());

pre_result = median_predam.reduceRegion(ee.Reducer.toList(), fc, 40).get('waterClass_median')
pre_medians = pre_result.getInfo()

pre_medians_dict = dict(Counter(pre_medians))
pre_medians_count = sorted(list(pre_medians_dict.items()), key = lambda x: x[0])

post_result = median_postdam.reduceRegion(ee.Reducer.toList(), fc, 40).get('waterClass_median')
post_medians = post_result.getInfo()

post_medians_dict = dict(Counter(post_medians))
post_medians_count = sorted(list(post_medians_dict.items()), key = lambda x: x[0])

counts = [pre_medians_count, post_medians_count]
print(counts)


In [ ]:
#sample data
pre_medians_count = [(1, 2234),(2, 23),(3,321)]
post_medians_count = [(1, 98),(1.5, 97),(2, 897),(2.5, 7),(3, 998)]

In [ ]:
columns = ['feature_ID',
           'pre_1.0','pre_1.5','pre_2.0','pre_2.5','pre_3.0',
           'post_1.0','post_1.5','post_2.0','post_2.5','post_3.0']

df = pd.DataFrame(columns = columns)
pre_indices = [x[0] for x in pre_medians_count]
post_indices = [x[0] for x in post_medians_count]

full_indices = [1, 1.5, 2, 2.5, 3]

missing_pre = list(set(full_indices).difference(pre_indices))
for i in missing_pre:
  empty_set = (i, 0)
  pre_medians_count.append(empty_set)

missing_post = list(set(full_indices).difference(post_indices))
for i in missing_post:
  empty_set = (i, 0)
  post_medians_count.append(empty_set)

pre_medians_count = sorted(pre_medians_count, key = lambda x: x[0])
post_medians_count = sorted(post_medians_count, key = lambda x: x[0])
print(pre_medians_count, post_medians_count)

#pre = [x[1] for x in pre_medians_count]
#post = [x[1] for x in post_medians_count]

#add = [1] + pre + post
#add = pd.DataFrame([add], columns = columns)

#df = pd.concat([df, add])
#df

In [ ]:
df

In [ ]:
#create "before dam" and "after dam" GSW data from intervals
range_predam = yearly_processed.filterDate(before_start,before_end)
range_postdam = yearly_processed.filterDate(after_start,after_end)

In [ ]:
#calculate median water coverage of "before dam" and "after dam"
median_predam = range_predam.reduce(ee.Reducer.median());
median_postdam = range_postdam.reduce(ee.Reducer.median());

In [ ]:
#catchment area bounds
bounds = ee.FeatureCollection("users/xg2180/Ataturk_catchment");

In [ ]:
#get list of median water pixels for "before" period
#pre_result = median_predam.reduceRegion(ee.Reducer.toList(), bounds, 30, bestEffort = True).get('waterClass_median')
pre_result = median_predam.reduceRegion(ee.Reducer.toList(), bounds, 40).get('waterClass_median')
pre_medians = pre_result.getInfo()

In [ ]:
#obtain count of each pixel value in "before" period
pre_medians_count = dict(Counter(pre_medians))
print(pre_medians_count)

In [ ]:
#get list of median water pixels for "after" period
#post_result = median_postdam.reduceRegion(ee.Reducer.toList(), bounds, 30, bestEffort = True).get('waterClass_median')
post_result = median_postdam.reduceRegion(ee.Reducer.toList(), bounds, 40).get('waterClass_median')
post_medians = post_result.getInfo()

In [ ]:
#obtain count of each pixel value in "after" period
post_medians_count = dict(Counter(post_medians))
print(post_medians_count)